In [0]:
dbutils.widgets.removeAll()

## REQUESTER NAME

In [0]:
dbutils.widgets.text("01_Requester_Name", "Enter_Your_Name", label="01 Requester Name")
requester = dbutils.widgets.get("01_Requester_Name")
requester = requester.strip().replace(" ", "_").upper()
requester 
#dbutils.widgets.remove("Requester_Name")

Out[7]: 'XZHANG'

In [0]:
# requester = "XZHANG"

## SCHEMA

In [0]:
dbutils.widgets.text("02_SF_Schema", "ADVANCEDANALYTICS_SANDBOX", label="02 SF Schema")
db_schema = dbutils.widgets.get("02_SF_Schema")
db_schema
#dbutils.widgets.remove("SCHEMA")

Out[9]: 'ADVANCEDANALYTICS_SANDBOX'

In [0]:
# db_schema = "ADVANCEDANALYTICS_SANDBOX"

## AUDIENCE TABLE NAMES

In [0]:
dbutils.widgets.text("04_Table_Names", "Enter_Table_Name", label="04 Table Names")
table_names = dbutils.widgets.get("04_Table_Names")
table_names = table_names.split(",")
table_names = [segment.strip() for segment in table_names]
table_names
#dbutils.widgets.remove("Table_Names")

Out[13]: ['CORE_SEED_MID24',
 'HIGH_MODERATE_SEED_MID24',
 'LOW_MODERATE_SEED_MID24',
 'LIGHT_SEED_MID24',
 'CBS_MID_2024_DEDUPED_HISPANIC_VIEWERS',
 'CBS_MID_2024_DEDUPED_AFRICAN_AMERICAN_VIEWER',
 'CBS_MID_2024_DEDUPED_STREAMING_VIEWERS']

## AUDIENCE NAMES

In [0]:
dbutils.widgets.text("05_Audience_Names", "Enter_Audience_Names", label="05 Audience Names")
audience_names = dbutils.widgets.get("05_Audience_Names")
audience_names = audience_names.split(",")
audience_names = [segment.strip().replace(" ", "_").upper() for segment in audience_names]
audience_names
#dbutils.widgets.remove("audience_names")

Out[16]: ['CORE',
 'HIGH_MODERATE',
 'LOW_MODERATE',
 'LIGHT',
 'HISPANIC',
 'AFRICAN_AMERICAN',
 'STREAMING']

## OUTPUT FILE NAME

In [0]:
dbutils.widgets.text("03_Output_Name", "Enter_Output_Name", label="03 Output Name")
output_name = dbutils.widgets.get("03_Output_Name")
output_name = output_name.strip().replace(" ", "_").upper()
output_name 

Out[19]: 'CBS_AUDIENCE'

## MINIMUM BAR SIZE IN PLOT

In [0]:
dbutils.widgets.text("06_Min_Bar_Size", "0", label="06 Minimum Bar Size")
min_bar_size_working = dbutils.widgets.get("06_Min_Bar_Size")
min_bar_size = int(min_bar_size_working) if int(min_bar_size_working) > 0 else 0
# min_bar_size = int(min_bar_size)
min_bar_size 

Out[20]: 0

## ID TYPE: BLU_ID vs. INSCAPE_TV_KEY

In [0]:
dbutils.widgets.text("07_ID_Type", "BLU_ID", label="07 ID Type")
id_type = dbutils.widgets.get("07_ID_Type")
id_type = id_type.strip().upper()
# id_type_working = dbutils.widgets.get("07_ID_Type")
# id_type = 'BLU_ID' if id_type_working.upper() == 'BLU_ID' else id_type_working.upper()
id_type

Out[23]: 'INSCAPE_TV_KEY'

In [0]:
# # plot image name
# output_name = 'TEST AUDIENCES'
# output_name = output_name.replace(" ", "_")
# output_name

In [0]:
!pip install --upgrade pip

DEPRECATION: distro-info 0.23ubuntu1 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [0]:
import os
# import snowflake.connector
import pandas as pd
import numpy as np
# import datetime
from datetime import date
from datetime import datetime
from functools import reduce

In [0]:
# Check if upsetplot is not installed, install it, otherwise skip it
import subprocess
import sys

reqs = subprocess.check_output([sys.executable, '-m', 'pip', 'freeze'])
installed_packages = [r.decode().split('==')[0] for r in reqs.split()]

if not(installed_packages.count('UpSetPlot') > 0 ):
    !pip install upsetplot
else:
    print('UpSetPlot is already installed.')

In [0]:
# !pip install upsetplot
import upsetplot
from upsetplot import generate_counts
from matplotlib import pyplot as plt
from upsetplot import generate_counts, plot
from upsetplot import UpSet
from upsetplot import from_memberships
# import snowflake.connector
from scipy import stats 

In [0]:
### DB - SPARK
from pyspark.sql.types import FloatType
from pyspark.sql.functions import col
from shutil import copyfile
### DB - PYTHON
import xlsxwriter
##############################################################################
## TO BE ABLE TO RUN A SCRIPT IN A REPO
from databricks.sdk.runtime import * 
#1.a
service_credential = dbutils.secrets.get(scope="Scope3",key="Scope1")
spark.conf.set("fs.azure.account.auth.type.hmimltestdatalake.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.hmimltestdatalake.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.hmimltestdatalake.dfs.core.windows.net", "fc8698e0-8d39-4c9c-9ff2-b8f1662c56ec")
spark.conf.set("fs.azure.account.oauth2.client.secret.hmimltestdatalake.dfs.core.windows.net", service_credential)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.hmimltestdatalake.dfs.core.windows.net", "https://login.microsoftonline.com/57045f6b-531e-46ed-9ed8-6a60e2869d82/oauth2/token")
##############################################################################
#1.b
sfOptions = {
"sfURL" : "hmimtd.us-east-1.snowflakecomputing.com",
"sfUser" : "svc_advancedanalytics_sandbox_knime",
"sfPassword" : "BzBgbuwtB&75g67^",
"sfDatabase" : "HORIZON_PROD",
"sfSchema" : "ADVANCEDANALYTICS_SANDBOX",
"sfWarehouse" : "MARKETING_SCIENCE_WH"
}

In [0]:
# Generate list of output image file names

image_names = ['01_UPSET_PLOT_SORTED_BY_CATEGORY_-_' , '02_UPSET_PLOT_SORTED_BY_VOLUME_-_' , 
               '03_UPSET_PLOT_SORTED_BY_CATEGORY_IN_THOUSAND_-_', '04_UPSET_PLOT_SORTED_BY_VOLUME_IN_THOUSAND_-_' ]
image_names = [(x + output_name + '.jpeg') for x in image_names ]

image_names

In [0]:
# Export Excel File Name:
excel_file_name = '00_UPSET_PLOT_OVERLAP_SIZE_-_' + output_name + '.xlsx'
excel_file_name = excel_file_name.replace(' ', '_') 
excel_file_name

In [0]:
File_initial_Path = os.getcwd()
File_initial_Path

In [0]:
## Snowflake Connection

In [0]:
audience_temp = {
'AUDIENCE' : table_names,
'AUDIENCE_SHORT_NAME': audience_names
}
audience_df = pd.DataFrame(audience_temp)
audience_df

In [0]:
# audience_temp = {
#     'AUDIENCE' :  ["H_BTB_ALL_WATER_IBTT_CS_ACTIVEIDS_L2Y_TABLE_LL_EXP",
#                     "H_BTB_FLAVORED_WATER_IBTT_CS_ACTIVEIDS_L2Y_TABLE_LL_EXP",
#                     "H_BTB_SPARKLING_WATER_IBTT_CS_ACTIVEIDS_L2Y_TABLE_LL_EXP",
#                     "H_BTB_UNFLAVORED_STILL_WATER_IBTT_CS_ACTIVEIDS_L2Y_TABLE_LL_EXP"
#                     ],
#     'AUDIENCE_SHORT_NAME' : ["ALL_WATER_BUYER",
#                             "FLAVORED_WATER_BUYER",
#                             "SPARKLING_WATER_BUYER",
#                             "UNFLAVORED_STILL_WATER_BUYER"
#                             ]
# }
# audience_df = pd.DataFrame(audience_temp)
# audience_df

In [0]:
# Generate working string 
# Step 1
audience_df['WORKING_STRING'] = """select """ + id_type + """, '""" + audience_df['AUDIENCE_SHORT_NAME'] + """' as VALUE
from HORIZON_PROD.""" + db_schema + """.""" + audience_df['AUDIENCE']
# audience_df['WORKING_STRING']

# Generate working string step 2
value_list = audience_df['WORKING_STRING']
# value_list

# step 3
value_list_2 = """
""" + value_list + """
"""
#value_list_2

seperator = """ 
union all 
"""
value_list_final = seperator.join(value_list_2)

# print(value_list_final)

In [0]:
# query string part 2
# Generate working string part2
audience_df['WORKING_STRING_P2'] = """MAX( CASE WHEN VALUE = '""" + audience_df['AUDIENCE_SHORT_NAME'] + """' THEN 1 ELSE 0 END) AS """ + '''"''' + audience_df['AUDIENCE_SHORT_NAME'] + '''"'''
# audience_df['WORKING_STRING_P2']

# Generate working string
value_list_p2 = audience_df['WORKING_STRING_P2']
#value_list_p2

seperator_p2 = """,
"""
value_list_final_p2 = seperator_p2.join(value_list_p2)

# print(value_list_final_p2)

In [0]:
# query string part 3
# Generate working string part3
audience_df['WORKING_STRING_P3'] = '''"''' + audience_df['AUDIENCE_SHORT_NAME'] + '''"'''
seperator_p3 = """, """
value_list_final_p3 = seperator_p3.join(audience_df['WORKING_STRING_P3'])

print(value_list_final_p3)

In [0]:
new_query = f"""
with wq as
(
    {value_list_final}
),
wq2 as
(
    select distinct {id_type}, VALUE
    from wq
),
wq3 as
(
    SELECT
    {id_type},
    {value_list_final_p2}
    FROM wq2
    group by {id_type}
)
select {value_list_final_p3}, count(distinct {id_type}) as SIZE from wq3
group by {value_list_final_p3}
"""

print(new_query)

In [0]:
df_to_return = spark.read.format("snowflake").options(**sfOptions).option("query", new_query).load()

In [0]:
df_to_return_pd = df_to_return.toPandas()

In [0]:
# remove doubble quotes from column headers
df_to_return_pd.columns = [col.replace('"', '') for col in df_to_return_pd.columns]

In [0]:
# Convert strings to integers
# df_to_return_pd['SIZE'] = df_to_return_pd['SIZE'].astype(str).astype(int)
df_to_return_pd = df_to_return_pd.applymap(lambda x: int(str(x)))

df_to_return_pd.dtypes

In [0]:
# Make a copy of the returned dataframe
df = df_to_return_pd.copy()
# df

In [0]:
# Convert 1s and 0s to True and False
# Replace 1 with True and 0 with False

cols_to_replace = df.columns[:len(audience_df['AUDIENCE_SHORT_NAME'])] 
cols_to_replace

df[cols_to_replace] = df[cols_to_replace].replace(1, True)
df[cols_to_replace] = df[cols_to_replace].replace(0, False)
df.head(10)

In [0]:
# reset index
df_2 = df.set_index(list(df.columns[:len(audience_df['AUDIENCE_SHORT_NAME'])]))

# convert dataframe to series
c_series = df_2.squeeze(axis=0)
# c_series

plot_series = c_series.SIZE
plot_series

In [0]:
# Check on the data type - Need to be Integer
plot_series.dtype

## Upset Plot v1: Raw Audience Sizes - Sorted by Category

In [0]:
### Upset Plot by Audience Name

# create plot sorted by category
cplot = UpSet(plot_series,sort_categories_by='cardinality', show_counts='%02d', show_percentages=True,element_size=50, min_subset_size = min_bar_size)
# cplot

# show plot
fig = plt.figure(figsize=(100, 200))
cplot.plot()

# set plot name
# lot_file_name = image_names[0]
# plot_file_name
temp_plot_file_name = "/tmp/"+ image_names[0]
# temp_plot_file_name

# 1. Save the image file
plt.savefig(temp_plot_file_name, dpi=199, bbox_inches="tight")

## Upset Plot v2: Raw Audience Sizes - Sorted by Volume

In [0]:
### Upset Plot by Audience Name

# create plot sorted by category
cplot2 = UpSet(plot_series, sort_by='cardinality', show_counts='%02d', show_percentages=True,element_size=50, min_subset_size = min_bar_size)

# show plot
fig = plt.figure(figsize=(100, 200))
cplot2.plot()

# set plot name
# plot_file_name = image_names[1]
# plot_file_name
temp_plot_file_name = "/tmp/"+ image_names[1]
# temp_plot_file_name

# 1. Save the image file
plt.savefig(temp_plot_file_name, dpi=199, bbox_inches="tight")

In [0]:
## Show audience size in thousands

# Divide size by 1000 to fit the plot
df_k = df.copy()
df_k['SIZE'] = (df_k['SIZE']/1000).round().astype(int)
# df_k.head()

# Renaming the columns
cols_to_modify = df_k.columns[:len(audience_df['AUDIENCE_SHORT_NAME'])] 
cols_to_modify

for col in cols_to_modify:
    df_k.rename(columns={col: col + " (K)"}, inplace=True)
# df_k.head(10)

# reset index
df_k_2 = df_k.set_index(list(df_k.columns[:len(audience_df['AUDIENCE_SHORT_NAME'])]))
#df_k_2

# convert dataframe to series
c_series_k = df_k_2.squeeze(axis=0)
# c_series_k

plot_series_k = c_series_k.SIZE
plot_series_k

In [0]:
# Min bar size by K
min_bar_size_k = round(min_bar_size / 1000 )
min_bar_size_k 

## Upset Plot v3: Audience Sizes by Thousands - Sorted by Category

In [0]:
### Upset Plot by Audience Name

# create plot sorted by category
cplot_k = UpSet(plot_series_k, sort_categories_by='cardinality', show_counts='%02d', show_percentages=True,element_size=50, min_subset_size = min_bar_size_k )
# cplot

# show plot
fig = plt.figure(figsize=(100, 200))
cplot_k.plot()

# set plot name
# lot_file_name = image_names[0]
# plot_file_name
temp_plot_file_name = "/tmp/"+ image_names[2]
# temp_plot_file_name

# 1. Save the image file
plt.savefig(temp_plot_file_name, dpi=199, bbox_inches="tight")

## Upset Plot v4: Audience Sizes by Thousands - Sorted by Volume

In [0]:
### Upset Plot by Audience Name

# create plot sorted by category
cplot_k2 = UpSet(plot_series_k,sort_by='cardinality', show_counts='%02d', show_percentages=True,element_size=50, min_subset_size = min_bar_size_k )

# show plot
fig = plt.figure(figsize=(100, 200))
cplot_k2.plot()

temp_plot_file_name = "/tmp/"+ image_names[3]
# temp_plot_file_name

# 1. Save the image file
plt.savefig(temp_plot_file_name, dpi=199, bbox_inches="tight")

In [0]:
current_date = date.today().strftime("%Y_%m_%d")
current_date

In [0]:
current_time = datetime.now().strftime("%H_%M_%S")
current_time

In [0]:

# Copy data from temp storage into dbfs storage, then move the file into Azure Blob storage
azure_path = "abfss://marketing-science@hmimltestdatalake.dfs.core.windows.net/ROUTINE_SCRIPTS_OUTPUTS/UPSET_PLOT/"

for x in image_names:

    # 2. Copy data from temp storage into dbfs storage 
    dbfs_data_path = f"""/dbfs/mnt/{x}"""
    temp_plot_file_name_2 = "/tmp/" + x
    copyfile(temp_plot_file_name_2 , dbfs_data_path)

    # 3. Move plot image file from dbfs storage into Azure Blob storage
    # azure_file_name = azure_path + f"{requester}/{x}"
    azure_file_name = azure_path + f"{requester}/{current_date}/{current_time}/{x}"
    # print(azure_file_name)
    dbfs_data_path_for_moving = dbfs_data_path.replace("/dbfs", "dbfs:")
    # print(dbfs_data_path_for_moving)
    dbutils.fs.mv(dbfs_data_path_for_moving, azure_file_name)

    print(x, "is saved into Azure Storage")

In [0]:
# Export Excel file:
# 1. Save Excel file: 
temp_excel_file_name = "/tmp/"+ excel_file_name
df_to_return_pd.to_excel(temp_excel_file_name, index = False, sheet_name = 'audience overlap')

# 2. Copy data from temp storage into dbfs storage 
dbfs_data_path = f"""/dbfs/mnt/{excel_file_name}"""
temp_excel_file_name_2 = "/tmp/" + excel_file_name
copyfile(temp_excel_file_name_2 , dbfs_data_path)

# 3. Move export file from dbfs storage into Azure Blob storage
# azure_file_name = azure_path + f"{requester}/{excel_file_name}"
azure_file_name = azure_path + f"{requester}/{current_date}/{current_time}/{excel_file_name}"
# print(azure_file_name)
dbfs_data_path_for_moving = dbfs_data_path.replace("/dbfs", "dbfs:")
# print(dbfs_data_path_for_moving)
dbutils.fs.mv(dbfs_data_path_for_moving, azure_file_name)

print(excel_file_name, "is saved into Azure Storage")

In [0]:
print("Done with Upset Plots!")